### Word level text generation + Embedding vectors

In this notebook we are going to learn how load pretrained word vectors to our model for text generation.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Imports


In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras

import numpy as np
import tensorflow as tf
import pandas as pd

import os, time, math, string, random

np.__version__

'1.19.5'

#### Loading the data

In [3]:
base_path = '/content/drive/My Drive/NLP Data/text-gen/New York Times Comments'
os.path.exists(base_path)

True

In [4]:
%%time
all_headlines = []
for filename in os.listdir(base_path):
  if 'Articles' in filename:
    df = pd.read_csv(os.path.join(base_path, filename))
    all_headlines.extend(list(
        df.headline.values
    ))
    break
all_headlines = [h for h in all_headlines if h != "Unknown"]
print(len(all_headlines))

1214
CPU times: user 14.1 ms, sys: 6.89 ms, total: 20.9 ms
Wall time: 28.1 ms


In [5]:
all_headlines[:15]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?',
 'Commuter Reprogramming',
 'Ford Changed Leaders, Looking for a Lift. It’s Still Looking.',
 'Romney Failed to Win at Utah Convention, But Few Believe He’s Doomed',
 'Chain Reaction',
 'He Forced the Vatican to Investigate Sex Abuse. Now He’s Meeting With Pope Francis.',
 'In Berlin, artists find a home',
 'The Right Stuff',
 'Jimmy Carter Knows What North Korea Wants',
 'The Truth Is Out There',
 'New Jersey Ruling Could Reignite Battle Over Church-State Separation']

### Data preparation

The first step that we are going to do for data preperation is text-clening. We are going to remove punctuations and lowercase all words.

In [7]:
def clean_text(txt):
  txt = "".join(v for v in txt if v not in string.punctuation).lower()
  txt = txt.encode('utf8').decode('ascii', 'ignore')
  return txt
corpus = [clean_text(x) for x in all_headlines]
random.shuffle(corpus)
corpus[:10]

['childhood fears no parent can allay',
 'variety acrostic',
 'teaching activities for i spy at new yorks museum of deception',
 'how strenuous exercise affects our immune system',
 'voter suppressions legacy lives on',
 'us set to blunt pollution rules for automakers',
 'the americans season 6 episode 4 recap the birth of a honey pot',
 'perus love affair with the potato',
 'how to make facebook more accountable',
 'why your bank account interest is still paltry']

### Generating Sequence of N-grams Tokens
Language modelling requires a sequence input data, as given a sequence of word-tokens the aim is to predict the next word.

### Tokenization

In [8]:
tokenizer = keras.preprocessing.text.Tokenizer()

def get_sequence_of_tokens(corpus):
  tokenizer.fit_on_texts(corpus)
  total_words = len(tokenizer.word_index) + 1
  input_sequences = []
  for line in corpus:
    token_list= tokenizer.texts_to_sequences([
        line
    ])[0]
    for i in range(1, len(token_list)):
      n_gram_sequence = token_list[:i+1]
      input_sequences.append(n_gram_sequence)
  return input_sequences, total_words

In [9]:
inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:28]

[[582, 269],
 [582, 269, 36],
 [582, 269, 36, 1115],
 [582, 269, 36, 1115, 27],
 [582, 269, 36, 1115, 27, 1116],
 [270, 371],
 [31, 47],
 [31, 47, 6],
 [31, 47, 6, 59],
 [31, 47, 6, 59, 372],
 [31, 47, 6, 59, 372, 13],
 [31, 47, 6, 59, 372, 13, 14],
 [31, 47, 6, 59, 372, 13, 14, 583],
 [31, 47, 6, 59, 372, 13, 14, 583, 1117],
 [31, 47, 6, 59, 372, 13, 14, 583, 1117, 4],
 [31, 47, 6, 59, 372, 13, 14, 583, 1117, 4, 1118],
 [15, 1119],
 [15, 1119, 584],
 [15, 1119, 584, 1120],
 [15, 1119, 584, 1120, 66],
 [15, 1119, 584, 1120, 66, 585],
 [15, 1119, 584, 1120, 66, 585, 1121],
 [1122, 1123],
 [1122, 1123, 586],
 [1122, 1123, 586, 373],
 [1122, 1123, 586, 373, 8],
 [22, 587],
 [22, 587, 3]]

### How ngrams works?

```
[1, 12] -> this is
[1, 12, 5170] -> this is a
[1, 12, 5170, 3366]-> this is a dog
```

### Loading the pretrain word vectors.

In [10]:
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [11]:
embedding_path = "/content/drive/MyDrive/NLP Data/glove.6B/glove.6B.100d.txt"

In [12]:
embeddings_dictionary = dict()
start = time.time()
with open(embedding_path, encoding='utf8') as glove_file:
    for line in glove_file:
        records = line.split()
        word  = records[0]
        vectors = np.asarray(records[1:], dtype='float32')
        embeddings_dictionary[word] = vectors

print(f"ETA: {hms_string(time.time() - start)}")

ETA: 0:00:11.62


Prepare embedding vectors to suit our own data.

In [13]:
start = time.time()
embedding_matrix = np.zeros((total_words, 100))
for word, index in tokenizer.word_index.items():
    vector = embeddings_dictionary.get(word)
    if vector is not None:
      embedding_matrix[index] = vector
    
print(f"ETA: {hms_string(time.time() - start)}")

ETA: 0:00:00.01


### Padding sequences and generating target.

Sequences must be of the same length. Ww will create n-grams sequences as predictors and the next word of the n-grams as label.

```
they -> are (label)
they are -> standing (label)
they are standing -> at (label)
they are standing at -> the (label)
they are standing at the -> door (label)
```

In [14]:
def generate_padded_sequences(input_sequences):
  max_sequence_len = max([len(x) for x in input_sequences])
  input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, 
                                            padding='pre'))
  predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
  label = keras.utils.to_categorical(label, num_classes=total_words)
  return predictors, label, max_sequence_len

In [15]:
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

### LSTM for text generation

Unlike Feed-forward neural networks in which activation outputs are propagated only in one direction, the activation outputs from neurons propagate in both directions (from inputs to outputs and from outputs to inputs) in Recurrent Neural Networks. This creates loops in the neural network architecture which acts as a ‘memory state’ of the neurons. This state allows the neurons an ability to remember what have been learned so far.

The memory state in RNNs gives an advantage over traditional neural networks but a problem called Vanishing Gradient is associated with them. In this problem, while learning with a large number of layers, it becomes really hard for the network to learn and tune the parameters of the earlier layers. To address this problem, A new type of RNNs called LSTMs (Long Short Term Memory) Models have been developed.

LSTMs have an additional state called ‘cell state’ through which the network makes adjustments in the information flow. The advantage of this state is that the model can remember or forget the leanings more selectively. To learn more about LSTMs, here is a great post. Lets architecture a LSTM model in our code.



In [16]:
predictors

array([[   0,    0,    0, ...,    0,    0,  582],
       [   0,    0,    0, ...,    0,  582,  269],
       [   0,    0,    0, ...,  582,  269,   36],
       ...,
       [   0,    0,    0, ...,    0, 3492,  439],
       [   0,    0,    0, ...,    0,    0,  207],
       [   0,    0,    0, ...,    0,  207, 3493]], dtype=int32)

In [22]:
def create_model(max_sequence_len, total_words):
  input_len = max_sequence_len - 1
  model = keras.Sequential(name="text-generator")
  model.add(keras.layers.Embedding( total_words, 100,
                                    weights=[embedding_matrix], 
                                    trainable=True,
                                    input_length=input_len))
  model.add(keras.layers.Bidirectional(keras.layers.LSTM(100, return_sequences=True)))
  model.add(keras.layers.LSTM(100))
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Dense(512, activation='relu'))
  model.add(keras.layers.Dense(512, activation='relu'))
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Dense(total_words, activation='softmax'))
  model.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=["acc"])
  return model
model = create_model(max_sequence_len, total_words)
model.summary()

Model: "text-generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 23, 100)           349400    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 23, 200)           160800    
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               51712     
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)            

In [24]:
model.fit(predictors, label,
          epochs=100, verbose=1
          )

Epoch 1/100
244/244 [==============================] - 7s 30ms/step - loss: 7.0916 - acc: 0.0292
Epoch 2/100
244/244 [==============================] - 7s 30ms/step - loss: 6.9852 - acc: 0.0324
Epoch 3/100
244/244 [==============================] - 7s 30ms/step - loss: 6.8692 - acc: 0.0364
Epoch 4/100
244/244 [==============================] - 7s 30ms/step - loss: 6.7432 - acc: 0.0404
Epoch 5/100
244/244 [==============================] - 7s 30ms/step - loss: 6.6500 - acc: 0.0452
Epoch 6/100
244/244 [==============================] - 7s 30ms/step - loss: 6.5323 - acc: 0.0458
Epoch 7/100
244/244 [==============================] - 7s 30ms/step - loss: 6.4127 - acc: 0.0506
Epoch 8/100
244/244 [==============================] - 7s 30ms/step - loss: 6.2904 - acc: 0.0545
Epoch 9/100
244/244 [==============================] - 7s 30ms/step - loss: 6.1625 - acc: 0.0596
Epoch 10/100
244/244 [==============================] - 7s 30ms/step - loss: 6.0521 - acc: 0.0672
Epoch 11/100
244/244 [=======

### Generating text.
Great, our model architecture is now ready and we can train it using our data. Next lets write the function to predict the next word based on the input words (or seed text). We will first tokenize the seed text, pad the sequences and pass into the trained model to get predicted word. The multiple predicted words can be appended together to get predicted sequence.

In [25]:
def generate_text(seed_text, max_len, model, max_sequence_len):
  for _ in range(max_len):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list],
                               maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list)
    predicted = np.argmax(predicted, axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
  return seed_text.title()

In [26]:
print (generate_text("united states", 5, model, max_sequence_len))
print (generate_text("preident trump", 4, model, max_sequence_len))
print (generate_text("donald trump", 4, model, max_sequence_len))
print (generate_text("india and china", 4, model, max_sequence_len))
print (generate_text("new york", 4, model, max_sequence_len))
print (generate_text("science and technology", 5, model, max_sequence_len))

United States Are Doing What Scott Pruitt
Preident Trump Blasts Comey In Barrage
Donald Trump Blasts Comey In Barrage
India And China Triumph Of Liang Populist
New York City Decline Crisis Plot
Science And Technology Video To Close Back At
